### The CAPM

The Capital Asset Pricing Model, or CAPM, holds a central position in the field of finance by allowing for the estimation of the expected return of a financial asset based on its level of risk. Its principle is grounded in the notion that investors demand an additional risk premium above the risk-free rate to compensate for the inherent risk associated with a specific investment. This risk premium is determined by considering the correlation between the asset's return and that of the overall market, as well as the risk-free rate and market volatility. Through these elements, the CAPM provides an estimate of the expected return of an asset based on its systematic risk, thus offering a valuable analytical framework for investment decision-making.

In practice, I utilize the CAPM to assess the anticipated evolution of each stock's price over a given period. The formula for the CAPM is as follows:
$Ri = rf + ß*(Rm-rf)$

Where $Ri$ represents the expected return of asset $i$, $rf$ is the risk-free rate, $ß$ symbolizes the correlation between the market and asset $i$, and $Rm$ represents the market return.    
The coefficient $ß$ is calculated as follows:
$ß = Cov(Market, Asset) / Var(Market)$    
I have chosen to measure the covariance between the market at time $n$ and asset $i$ at time $n+1$ to better understand the correlation of market fluctuations with asset $i$ price at that particular moment.

In the final DataFrame, the "CAPM" column indicates the expected return of the stock. It needs to be multiplied by 100 to express it as a percentage.

### Rules for the inputs : 

**rf** : Define the risk-free rate (default 3%)    
**arg** : Define the study period in years (default 5 years) and the Investment horizon interval: Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]    
**market** : Define the benchmark (default is CAC40)    
**ticker_list** : Define the companies to analyze

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [ ]:
rf = 0.03
arg = {"period": "5y", "interval": "1wk"}
market = "^FCHI"
ticker_list = ["AI.PA", "AIR.PA", "TTE.PA", "DEC.PA", "ALO.PA", "GLE.PA","MT.AS", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA", "CA.PA", "ACA.PA", "BN.PA", "DSY.PA", "ENGI.PA", "EL.PA", "ERF.PA", "RMS.PA", "KER.PA","LR.PA", "OR.PA", "MC.PA", "ML.PA", "ORA.PA", "RI.PA", "PUB.PA", "RNO.PA", "SAF.PA", "SGO.PA", "SAN.PA", "SU.PA", "STLAP.PA", "STMPA.PA", "TEP.PA", "HO.PA", "VIE.PA", "DG.PA", "WLN.PA"]
ticker_list.append(market)

In [6]:
# Initializing the df_variation dataframe to collect all the variations of each of the studied companies
df_variation = pd.DataFrame()
for company in ticker_list: 
    # Retrieving the weekly prices of the stock 
    stock = yf.Ticker(company)
    data = stock.history(**arg)

    # Converting prices into variations and storing them in the "df_variation" dataframe
    variation = data['Close'].pct_change().dropna()
    df_variation = pd.concat([df_variation, variation], axis=1)
    df_variation.rename(columns={'Close': company}, inplace=True)

In [8]:
#I shift by 1 to calculate the correlation between the market at time t and the movement of the stock the next day at time t+1
df_variation[market] = df_variation[market].shift(1)

# Calculating the CAPM
df_CAPM = pd.DataFrame()
df_CAPM["Beta"] = df_variation.cov()[market] / df_variation[market].var()
df_CAPM["CAPM"] = rf + df_CAPM["Beta"] * ( df_variation[market].mean() - rf )
df_CAPM.sort_values(by="CAPM")

,Beta,CAPM
^FCHI,1.000000,0.002075
STLAP.PA,0.257574,0.022807
ALO.PA,0.227028,0.023660
MT.AS,0.194426,0.024571
STMPA.PA,0.183862,0.024866
EN.PA,0.178380,0.025019
ACA.PA,0.173916,0.025143
AIR.PA,0.149230,0.025833
PUB.PA,0.134848,0.026234
EL.PA,0.103606,0.027107
